In [ ]:
import os

from dotenv import load_dotenv

load_dotenv(os.path.join("..", ".env"), override=True)

%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="LangSmith now uses UUID v7", 
    category=UserWarning,
)

# 6. Production Deep Agent — Prompt Caching + DynamoDB

## Overview

This notebook takes the enhanced agent from Notebook 5 and adds **production-ready persistence**:

| Feature | In-memory (Notebooks 1-5) | Production (This Notebook) |
|---------|--------------------------|---------------------------|
| **Chat History** | Lost on restart | ✅ Persisted in DynamoDB |
| **TODOs** | RAM only | ✅ Persisted in DynamoDB |
| **Files** | RAM only | ✅ Persisted in DynamoDB |
| **Prompt Cost** | Full price every turn | ✅ 90% savings with caching |
| **Latency** | Reprocess entire prompt | ✅ 85% faster TTFT |

### Architecture

```
┌──────────────────────────────────────────────────────┐
│                   Orchestrator Agent                  │
│  ┌─────────────┐  ┌──────────┐  ┌────────────────┐  │
│  │ Cached Prompt│  │ DynamoDB │  │  DynamoDB      │  │
│  │ (Nova Cache) │  │ State    │  │  Artifacts     │  │
│  │  90% savings │  │ (Thread) │  │  (TODOs/Files) │  │
│  └─────────────┘  └──────────┘  └────────────────┘  │
│                                                      │
│  Tools: dynamo_write/read_todos, dynamo_write/read   │
│         _file, dynamo_ls, edit_file, glob_files,     │
│         grep_files, load_skill, task                 │
│                               │                      │
│                    ┌──────────┴──────────┐           │
│                    │  Research Sub-agent  │           │
│                    │  tavily_search +     │           │
│                    │  think_tool          │           │
│                    └─────────────────────┘           │
└──────────────────────────────────────────────────────┘
```

---

## Part 1: Prompt Caching for Nova

### How it works

Amazon Nova supports **prompt caching** to avoid re-processing the same prompt prefix on every turn:

| Type | How | Savings | Predictability |
|------|-----|---------|----------------|
| **Implicit** | Automatic by Nova | ~90% | Non-deterministic |
| **Explicit** | `cachePoint` marker | 90% | ✅ Deterministic |

### Key Rule: Prompt Ordering

```
┌──────────────────────────────────────────┐
│  STATIC CONTENT (cached, pay 10%)        │
│  ├── Base instructions                   │
│  ├── Tool usage rules                    │
│  └── Skills discovery listing            │
├──────────── cachePoint ──────────────────│
│  DYNAMIC CONTENT (re-processed, 100%)    │
│  ├── Today's date                        │
│  └── Session-specific context            │
└──────────────────────────────────────────┘
```

Everything **before** the cache point is cached and reused. Everything **after** is re-processed each turn.

In [ ]:
from langchain_aws import ChatBedrockConverse

# --- The built-in cache point method ---
cache_point = ChatBedrockConverse.create_cache_point()
print("Cache point structure:", cache_point)
# Output: {'cachePoint': {'type': 'default'}}

# This goes into SystemMessage.additional_kwargs

In [ ]:
from deep_agents_from_scratch.prompt_caching import (
    create_cached_system_message,
    build_cached_prompt,
    estimate_cache_savings,
)

# --- Demo: Create a cached system message ---
cached_msg = create_cached_system_message(
    static_content="You are a research agent. You have access to tools for web search, file management, and task planning.",
    dynamic_content="Today's date is Feb 11, 2026."
)

print("SystemMessage content:")
print(cached_msg.content[:200])
print()
print("additional_kwargs (cache marker):")
print(cached_msg.additional_kwargs)
# {'cachePoint': {'type': 'default'}}

In [ ]:
# --- Estimate cost savings ---
# Assume a typical system prompt of ~5000 tokens
savings = estimate_cache_savings(
    prompt_tokens=5000,
    turns_per_session=10,
    cost_per_1k_input=0.0008,  # Nova Pro pricing
)

print("📊 Cache Savings Estimate:")
print(f"   Prompt tokens: {savings['prompt_tokens']}")
print(f"   Turns/session: {savings['turns']}")
print(f"   Without cache: ${savings['uncached_cost']:.4f}")
print(f"   With cache:    ${savings['cached_cost']:.4f}")
print(f"   Savings:       ${savings['savings']:.4f} ({savings['savings_percent']}%)")

---

## Part 2: DynamoDB Setup

### Table Design

| Table | PK | SK | Purpose |
|-------|----|----|--------|
| `DeepAgents_State` | `pk` (thread_id) | `sk` (checkpoint_id) | LangGraph state persistence |
| `DeepAgents_Artifacts` | `thread_id` | `artifact_id` | Persistent TODOs and Files |

### Artifact ID Patterns

| Artifact Type | artifact_id format | Example |
|---------------|-------------------|--------|
| TODO list | `TODO#LIST` | `TODO#LIST` |
| File | `FILE#<path>` | `FILE#findings_mcp.md` |
| Skill | `FILE#/skills/<name>/SKILL.md` | `FILE#/skills/web-research/SKILL.md` |

In [ ]:
from deep_agents_from_scratch.checkpoint_dynamo import (
    create_dynamodb_tables,
    wait_for_tables,
    get_checkpointer,
    DEFAULT_STATE_TABLE,
    DEFAULT_ARTIFACTS_TABLE,
)

# --- Create tables (idempotent — safe to run multiple times) ---
results = create_dynamodb_tables(region_name="us-east-1")
print("Table creation results:")
for table, status in results.items():
    print(f"  {table}: {status}")

# Wait for tables to be ready
wait_for_tables(region_name="us-east-1")

In [ ]:
# --- Get checkpointer ---
checkpointer = get_checkpointer(
    table_name=DEFAULT_STATE_TABLE,
    region_name="us-east-1",
)
print(f"Checkpointer type: {type(checkpointer).__name__}")

---

## Part 3: Persistent Tools

### DynamoDB-backed vs In-memory Tools

| In-memory (Notebook 5) | DynamoDB (This notebook) | Difference |
|----------------------|-------------------------|------------|
| `write_todos` | `dynamo_write_todos` | Persists to DynamoDB |
| `read_todos` | `dynamo_read_todos` | Reads from DynamoDB |
| `write_file` | `dynamo_write_file` | Dual-write: memory + DynamoDB |
| `read_file` | `dynamo_read_file` | Memory-first, DynamoDB fallback |
| `ls` | `dynamo_ls` | Merges memory + DynamoDB files |

In [ ]:
from deep_agents_from_scratch.dynamo_tools import (
    dynamo_write_todos,
    dynamo_read_todos,
    dynamo_write_file,
    dynamo_read_file,
    dynamo_ls,
)

# Preview the persistent tools
persistent_tools = [dynamo_write_todos, dynamo_read_todos, dynamo_write_file, dynamo_read_file, dynamo_ls]
print("📦 Persistent DynamoDB Tools:")
for t in persistent_tools:
    print(f"  🔧 {t.name}: {t.description[:80]}...")

---

## Part 4: Build the Production Agent

In [ ]:
from datetime import datetime
from langchain_aws import ChatBedrockConverse

from deep_agents_from_scratch.state import DeepAgentState
from deep_agents_from_scratch.enhanced_file_tools import edit_file, glob_files, grep_files
from deep_agents_from_scratch.skills import load_skill, RESEARCH_SKILL_MD, CODE_REVIEW_SKILL_MD
from deep_agents_from_scratch.research_tools import tavily_search, think_tool
from deep_agents_from_scratch.task_tool import _create_task_tool, SubAgent
from deep_agents_from_scratch.prompts import (
    RESEARCHER_INSTRUCTIONS,
    TODO_USAGE_INSTRUCTIONS,
    FILE_USAGE_INSTRUCTIONS,
    SUBAGENT_USAGE_INSTRUCTIONS,
)
from deep_agents_from_scratch.prompt_caching import build_cached_prompt

# --- Model ---
model = ChatBedrockConverse(
    model="us.amazon.nova-2-lite-v1:0",
    region_name="us-east-1",
    temperature=0.0,
)
print(f"✅ Model: {model.model}")

In [ ]:
# --- Orchestrator tools (DynamoDB-backed) ---
orchestrator_tools = [
    # Planning (persistent)
    dynamo_write_todos,
    dynamo_read_todos,
    # Files (persistent + enhanced)
    dynamo_ls,
    dynamo_read_file,
    dynamo_write_file,
    edit_file,
    glob_files,
    grep_files,
    # Skills
    load_skill,
    # Thinking
    think_tool,
]

# --- Sub-agents ---
subagents = [
    SubAgent(
        name="research-agent",
        description="Delegated research agent for complex web searches.",
        prompt=RESEARCHER_INSTRUCTIONS.format(date=datetime.now().strftime("%a %b %-d, %Y")),
        tools=["tavily_search", "think_tool"],
    ),
]

# Create task tool
all_tools_for_registry = orchestrator_tools + [tavily_search]
task_tool = _create_task_tool(
    tools=all_tools_for_registry,
    subagents=subagents,
    model=model,
    state_schema=DeepAgentState,
)

final_tools = orchestrator_tools + [task_tool]
print(f"✅ Tools: {len(final_tools)}")
for t in final_tools:
    print(f"   🔧 {t.name}")

In [ ]:
# --- Build Cached System Prompt ---
BASE_INSTRUCTIONS = """You are a highly capable AI assistant with planning, research, file management, and skills capabilities.
You operate in persistent mode — your TODOs and files are saved to DynamoDB and preserved across sessions.
When using todo or file tools, always pass the thread_id parameter."""

TOOL_INSTRUCTIONS = f"""{TODO_USAGE_INSTRUCTIONS}

{FILE_USAGE_INSTRUCTIONS}

{SUBAGENT_USAGE_INSTRUCTIONS.format(max_concurrent_research_units=3, max_researcher_iterations=3)}

## Skills System
You have access to a skills system. Skills are specialized instruction sets loaded from SKILL.md files.
- Use `dynamo_ls()` to discover available skills in the filesystem
- Use `load_skill(name)` to read full instructions when a skill is relevant
"""

DYNAMIC_CONTEXT = f"Today's date is {datetime.now().strftime('%A %B %-d, %Y')}."

# Build with optimal caching structure
cached_system_msg = build_cached_prompt(
    base_instructions=BASE_INSTRUCTIONS,
    tool_usage_instructions=TOOL_INSTRUCTIONS,
    dynamic_context=DYNAMIC_CONTEXT,
)

print(f"System prompt length: {len(cached_system_msg.content)} chars")
print(f"Cache point: {cached_system_msg.additional_kwargs}")

In [ ]:
from langchain.agents import create_agent

# --- Create the Production Agent (with DynamoDB checkpointer) ---
production_agent = create_agent(
    model,
    system_prompt=cached_system_msg.content,
    tools=final_tools,
    state_schema=DeepAgentState,
    checkpointer=checkpointer,  # DynamoDB persistence!
)

print("✅ Production agent created!")
print(f"   Model: {model.model}")
print(f"   Tools: {len(final_tools)}")
print(f"   Checkpointer: {type(checkpointer).__name__}")
print(f"   Persistence: DynamoDB")

---

## Part 5: Multi-Turn Conversation with Persistence

The key difference: with a DynamoDB checkpointer, we can:
- **Resume conversations** across sessions using the same `thread_id`
- **Persist TODOs and files** to DynamoDB for long-term storage
- **Share context** between agent invocations via thread-scoped artifacts

In [ ]:
import uuid
from utils import stream_agent

# Create a unique thread for this session
thread_id = f"session_{uuid.uuid4().hex[:8]}"
config = {"configurable": {"thread_id": thread_id}}

print(f"🧵 Thread ID: {thread_id}")
print(f"   This ID persists your conversation in DynamoDB.")
print(f"   Reuse it to resume later!")

In [ ]:
# --- Turn 1: Initial research request ---
initial_files = {
    "/skills/web-research/SKILL.md": RESEARCH_SKILL_MD,
    "/skills/code-review/SKILL.md": CODE_REVIEW_SKILL_MD,
}

query_1 = "Give me an overview of Model Context Protocol (MCP)."

result_1 = await stream_agent(
    production_agent,
    {
        "messages": [{"role": "user", "content": query_1}],
        "files": initial_files,
    },
    config=config,
)

In [ ]:
# View Turn 1 result
print("=" * 60)
print("TURN 1 RESPONSE")
print("=" * 60)
print(result_1["messages"][-1].content[:1000])
print()
print("📁 Files:", list(result_1.get("files", {}).keys())[:5])
print("📋 TODOs:", len(result_1.get("todos", [])))

In [ ]:
# --- Turn 2: Follow-up (agent remembers context from Turn 1) ---
query_2 = "Based on your research, what are the main security concerns with MCP?"

result_2 = await stream_agent(
    production_agent,
    {
        "messages": [{"role": "user", "content": query_2}],
    },
    config=config,  # Same thread_id = same conversation!
)

In [ ]:
# View Turn 2 result
print("=" * 60)
print("TURN 2 RESPONSE (uses context from Turn 1)")
print("=" * 60)
print(result_2["messages"][-1].content[:1000])

---

## Part 6: Verify DynamoDB Persistence

Let's directly query DynamoDB to confirm data was persisted.

In [ ]:
import boto3
from boto3.dynamodb.conditions import Key

dynamodb = boto3.resource("dynamodb", region_name="us-east-1")

# Check State table
state_table = dynamodb.Table("DeepAgents_State")
state_response = state_table.query(
    KeyConditionExpression=Key("pk").eq(thread_id),
    Limit=5,
)
print(f"📊 State table entries for thread '{thread_id}': {state_response['Count']}")

# Check Artifacts table
artifacts_table = dynamodb.Table("DeepAgents_Artifacts")
artifacts_response = artifacts_table.query(
    KeyConditionExpression=Key("thread_id").eq(thread_id),
)
print(f"📊 Artifacts for thread '{thread_id}':")
for item in artifacts_response.get("Items", []):
    artifact_id = item["artifact_id"]
    content_preview = item.get("content", "")[:80]
    print(f"   {artifact_id}: {content_preview}...")

---

## Summary: Production Architecture

### Prompt Caching
- `ChatBedrockConverse.create_cache_point()` → `{'cachePoint': {'type': 'default'}}`
- Place in `SystemMessage.additional_kwargs`
- Static content BEFORE cache point, dynamic AFTER
- **90% cost reduction** on system prompt tokens after Turn 1

### DynamoDB Persistence

| Component | Table | How |
|-----------|-------|-----|
| Chat history | `DeepAgents_State` | LangGraph checkpointer |
| TODOs | `DeepAgents_Artifacts` | `dynamo_write/read_todos` |
| Files | `DeepAgents_Artifacts` | `dynamo_write/read_file` |

### New Modules

| Module | Purpose |
|--------|--------|
| `prompt_caching.py` | Cache point utilities + cost estimation |
| `checkpoint_dynamo.py` | DynamoDB table creation + checkpointer |
| `dynamo_tools.py` | Persistent TODOs and Files |

### Next Steps
- Install `langgraph-checkpoint-dynamodb` for production-grade checkpointing
- Add `SummarizationMiddleware` equivalent for long conversations
- Implement `execute` tool with sandboxed execution (Modal, Runloop, or Daytona)